# Evaluation QAOA

In [ ]:
from mqt.problemsolver.partialcompiler import Partial_QAOA

In [ ]:
from mqt.problemsolver.partialcompiler import evaluator
for i in range(50,120,5):
    print("num qubits:", i)
    for j in range(2,10,3):
        print("num reps:", j)
        new, old = evaluator.evaluate_QAOA(num_qubits=i, repetitions=j)
        print(i, j, new[0]/old[0], new[1]/old[1], new[2]/old[2])